In [1]:
import pandas as pd
import numpy as np
import sqlite3

In [2]:
con = sqlite3.connect('../data/imdb/im.db')

In [3]:
imdb_schema = pd.read_sql("""

SELECT *
FROM sqlite_master

""", con)
imdb_schema

,type,name,tbl_name,rootpage,sql
0,table,movie_basics,movie_basics,2,"CREATE TABLE ""movie_basics"" (\n""movie_id"" TEXT..."
1,table,directors,directors,3,"CREATE TABLE ""directors"" (\n""movie_id"" TEXT,\n..."
2,table,known_for,known_for,4,"CREATE TABLE ""known_for"" (\n""person_id"" TEXT,\..."
3,table,movie_akas,movie_akas,5,"CREATE TABLE ""movie_akas"" (\n""movie_id"" TEXT,\..."
4,table,movie_ratings,movie_ratings,6,"CREATE TABLE ""movie_ratings"" (\n""movie_id"" TEX..."
5,table,persons,persons,7,"CREATE TABLE ""persons"" (\n""person_id"" TEXT,\n ..."
6,table,principals,principals,8,"CREATE TABLE ""principals"" (\n""movie_id"" TEXT,\..."
7,table,writers,writers,9,"CREATE TABLE ""writers"" (\n""movie_id"" TEXT,\n ..."


In [5]:
rating_info_comb_5k = pd.read_sql("""

SELECT *
FROM movie_basics AS b
    JOIN movie_ratings AS r
        ON b.movie_id = r.movie_id
WHERE numvotes > 5000

""", con)
rating_info_comb

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres,movie_id,averagerating,numvotes
0,tt1043726,The Legend of Hercules,The Legend of Hercules,2014,99.0,"Action,Adventure,Fantasy",tt1043726,4.2,50352
1,tt1171222,Baggage Claim,Baggage Claim,2013,96.0,Comedy,tt1171222,5.1,8296
2,tt1181840,Jack and the Cuckoo-Clock Heart,Jack et la mécanique du coeur,2013,94.0,"Adventure,Animation,Drama",tt1181840,7.0,5494
3,tt1210166,Moneyball,Moneyball,2011,133.0,"Biography,Drama,Sport",tt1210166,7.6,326657
4,tt1212419,Hereafter,Hereafter,2010,129.0,"Drama,Fantasy,Romance",tt1212419,6.5,87288
...,...,...,...,...,...,...,...,...,...
3916,tt7772580,The Perfection,The Perfection,2018,90.0,"Drama,Horror,Thriller",tt7772580,6.2,13573
3917,tt8041276,Paddleton,Paddleton,2019,89.0,"Comedy,Drama",tt8041276,7.2,5947
3918,tt8130968,Badla,Badla,2019,118.0,"Crime,Drama,Mystery",tt8130968,8.1,9988
3919,tt8439854,Lust Stories,Lust Stories,2018,120.0,"Drama,Romance",tt8439854,6.6,7691


In [7]:
tn_movie_budgets_df = pd.read_csv("../Data/tn.movie_budgets.csv")
tn_movie_budgets_df.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"


In [9]:
rating_info_comb.rename(columns={'primary_title':'movie'}, inplace=True)

In [10]:
info_budget_df = pd.merge(rating_info_comb, tn_movie_budgets_df, on='movie')

In [11]:
info_budget_df

,movie_id,movie,original_title,start_year,runtime_minutes,genres,movie_id,averagerating,numvotes,id,release_date,production_budget,domestic_gross,worldwide_gross
0,tt1043726,The Legend of Hercules,The Legend of Hercules,2014,99.0,"Action,Adventure,Fantasy",tt1043726,4.2,50352,42,"Jan 10, 2014","$70,000,000","$18,848,538","$58,953,319"
1,tt1171222,Baggage Claim,Baggage Claim,2013,96.0,Comedy,tt1171222,5.1,8296,38,"Sep 27, 2013","$8,500,000","$21,569,509","$22,885,836"
2,tt1210166,Moneyball,Moneyball,2011,133.0,"Biography,Drama,Sport",tt1210166,7.6,326657,15,"Sep 23, 2011","$50,000,000","$75,605,492","$111,300,835"
3,tt1212419,Hereafter,Hereafter,2010,129.0,"Drama,Fantasy,Romance",tt1212419,6.5,87288,61,"Oct 15, 2010","$50,000,000","$32,746,941","$108,660,270"
4,tt1232829,21 Jump Street,21 Jump Street,2012,109.0,"Action,Comedy,Crime",tt1232829,7.2,477771,44,"Mar 16, 2012","$42,000,000","$138,447,667","$202,812,429"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1523,tt2592614,Resident Evil: The Final Chapter,Resident Evil: The Final Chapter,2016,107.0,"Action,Horror,Sci-Fi",tt2592614,5.6,74979,84,"Jan 27, 2017","$40,000,000","$26,844,692","$314,101,190"
1524,tt2704998,Game Night,Game Night,2018,100.0,"Action,Comedy,Crime",tt2704998,7.0,163279,94,"Feb 23, 2018","$37,000,000","$69,179,066","$117,378,084"
1525,tt2980210,A Hologram for the King,A Hologram for the King,2016,98.0,"Comedy,Drama,Romance",tt2980210,6.1,36062,47,"Apr 22, 2016","$35,000,000","$4,212,494","$11,846,010"
1526,tt3125324,Beyond the Lights,Beyond the Lights,2014,116.0,"Drama,Music,Romance",tt3125324,6.9,14122,16,"Nov 14, 2014","$7,000,000","$14,618,727","$14,618,727"
